In [ ]:
import os
import gc
from glob import glob
import sys
import math
import time
import random
import shutil
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, List
from scipy.stats import entropy
from scipy.signal import butter, lfilter, freqz
from contextlib import contextmanager
from collections import defaultdict, Counter
# from kaggle_kl_div import score
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, log_loss
from tqdm import tqdm
from functools import partial
# import cv2
# from PIL import Image
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau, OneCycleLR, CosineAnnealingLR, CosineAnnealingWarmRestarts
from sklearn.preprocessing import LabelEncoder
from torchvision.transforms import v2
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import train_test_split
# import albumentations as A
# from albumentations import (Compose, Normalize, Resize, RandomResizedCrop, HorizontalFlip, VerticalFlip, ShiftScaleRotate, Transpose)
# from albumentations.pytorch import ToTensorV2
# from albumentations import ImageOnlyTransform
# import timm
import warnings 
warnings.filterwarnings('ignore')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from matplotlib import pyplot as plt
import joblib

In [ ]:
class CFG:
    version = 2
    criterion = nn.KLDivLoss(reduction="batchmean")
    num_workers = 1
    model_name = 'resnet1d_gru'
    optimizer='AdamW'
    epochs = 10
    lr = 1e-3
    batch_size = 64
    eval_every = 1000
    target_cols = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
    num_class = 6
    n_fold = 5
    # trn_fold = [0, 1, 2, 3, 4]
    trn_fold = [0, 1]
    PATH = '/kaggle/input/hms-harmful-brain-activity-classification/'
    EEG_FILE = '/kaggle/input/brain-eegs/eegs.npy'
    eeg_features = ['Fp1','T3','C3','O1','Fp2','C4','T4','O2']
    feature_to_index = {x:i for i, x in enumerate(eeg_features)}
    map_features = [
        ("Fp1", "T3"),
        ("T3", "O1"),
        ("Fp1", "C3"),
        ("C3", "O1"),
        ("Fp2", "C4"),
        ("C4", "O2"),
        ("Fp2", "T4"),
        ("T4", "O2"),
    ]
    num_features = len(map_features)
    in_channels = 8



In [ ]:
train = pd.read_csv(CFG.PATH + '/train.csv')
TARGETS = train.columns[-6:]
print(train.head())
print('Train shape:', train.shape )
print('Targets', list(TARGETS))

train['total_evaluators'] = train[TARGETS].sum(axis=1)

train_uniq = train.drop_duplicates(subset=['eeg_id'] + list(TARGETS))

print(f'There are {train.patient_id.nunique()} patients in the training data.')
print(f'There are {train.eeg_id.nunique()} EEG IDs in the training data.')
print(f'There are {train_uniq.shape[0]} unique eeg_id + votes in the training data.')

train_uniq.eeg_id.value_counts().value_counts().plot(kind='bar', title=f'Distribution of Count of EEG w Unique Vote: '
                                                                    f'{train_uniq.shape[0]} examples');
train = train_uniq.copy()
y_data = train[TARGETS].values +  1/6 # Regularization value
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

train['target'] = train['expert_consensus']
train = train.reset_index(drop=True)

del train_uniq
_ = gc.collect()

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(train['total_evaluators'], bins=10, color='blue', edgecolor='black')
plt.title('Histogram of Total Evaluators')
plt.xlabel('Total Evaluators')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
all_eegs = np.load(CFG.EEG_FILE, allow_pickle=True).item()
eeg_ids = train.eeg_id.unique()

In [ ]:
gkf = GroupKFold(n_splits=CFG.n_fold)

train["fold"] = -1

for fold_id, (_, val_idx) in enumerate(
    gkf.split(train, y=train["target"], groups=train["patient_id"])
):
    train.loc[val_idx, "fold"] = fold_id

In [ ]:
from scipy.signal import butter, lfilter

def quantize_data(data, classes):
    mu_x = mu_law_encoding(data, classes)
    return mu_x#quantized

def mu_law_encoding(data, mu):
    mu_x = np.sign(data) * np.log(1 + mu * np.abs(data)) / np.log(mu + 1)
    return mu_x

def mu_law_expansion(data, mu):
    s = np.sign(data) * (np.exp(np.abs(data) * np.log(mu + 1)) - 1) / mu
    return s

def butter_lowpass_filter(data, cutoff_freq=20, sampling_rate=200, order=4):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_freq / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = lfilter(b, a, data, axis=0)
    return filtered_data

class EEGDataset(Dataset):
    def __init__(
        self, df: pd.DataFrame, config, mode: str = 'train',
        eegs: Dict[int, np.ndarray] = all_eegs, downsample: int = None
    ): 
        self.df = df
        self.config = config
        self.mode = mode
        self.eegs = eegs
        self.downsample = downsample
        
    def __len__(self):
        """
        Length of dataset.
        """
        return len(self.df)
        
    def __getitem__(self, index):
        """
        Get one item.
        """
        X, y_prob = self.__data_generation(index)
        if self.downsample is not None:
            X = X[::self.downsample,:]
        output = {
            "eeg": torch.tensor(X, dtype=torch.float32),
            "labels": torch.tensor(y_prob, dtype=torch.float32)
        }
        return output
                        
    def __data_generation(self, index):
        row = self.df.iloc[index]
        X = np.zeros((10_000, CFG.num_features), dtype='float32')
        y = np.zeros(CFG.num_class, dtype='float32')
        data = self.eegs[row.eeg_id]

        # === Feature engineering ===
        feature_to_index = CFG.feature_to_index
        # X[:,0] = data[:,feature_to_index['Fp1']] - data[:,feature_to_index['T3']]
        # X[:,1] = data[:,feature_to_index['T3']] - data[:,feature_to_index['O1']]

        # X[:,2] = data[:,feature_to_index['Fp1']] - data[:,feature_to_index['C3']]
        # X[:,3] = data[:,feature_to_index['C3']] - data[:,feature_to_index['O1']]

        # X[:,4] = data[:,feature_to_index['Fp2']] - data[:,feature_to_index['C4']]
        # X[:,5] = data[:,feature_to_index['C4']] - data[:,feature_to_index['O2']]

        # X[:,6] = data[:,feature_to_index['Fp2']] - data[:,feature_to_index['T4']]
        # X[:,7] = data[:,feature_to_index['T4']] - data[:,feature_to_index['O2']]

        for i, p in enumerate(CFG.map_features):
            X[:, i] = data[:, feature_to_index[p[0]]] - data[:, feature_to_index[p[1]]]

        # === Standarize ===
        X = np.clip(X,-1024, 1024)
        X = np.nan_to_num(X, nan=0) / 32.0

        # === Butter Low-pass Filter ===
        X = butter_lowpass_filter(X)
        if self.mode != 'test':
            y_prob = row[self.config.target_cols].values.astype(np.float32)
        return X, y_prob

In [ ]:
train_dataset = EEGDataset(train, CFG, mode="train")
train_loader = DataLoader(
    train_dataset,
    batch_size=CFG.batch_size,
    shuffle=True,
    num_workers=CFG.num_workers, pin_memory=True, drop_last=True
)
output = train_dataset[0]
X, y = output["eeg"], output["labels"]
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

In [ ]:

class ResNet_1D_Block(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, downsampling, drop_prob):
        super(ResNet_1D_Block, self).__init__()
        self.bn1 = nn.BatchNorm1d(num_features=in_channels)
        self.relu = nn.ReLU(inplace=False)
        self.dropout = nn.Dropout(p=drop_prob, inplace=False)
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,
                               stride=stride, padding=padding, bias=False)
        self.bn2 = nn.BatchNorm1d(num_features=out_channels)
        self.conv2 = nn.Conv1d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size,
                               stride=stride, padding=padding, bias=False)
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.downsampling = downsampling

    def forward(self, x):
        identity = x

        out = self.bn1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)

        out = self.maxpool(out)
        identity = self.downsampling(x)

        out += identity
        return out


class EEGNet(nn.Module):

    def __init__(self, kernels, in_channels=20, fixed_kernel_size=17, num_classes=6, drop_prob=.1):
        super(EEGNet, self).__init__()
        self.kernels = kernels
        self.planes = 24
        self.parallel_conv = nn.ModuleList()
        self.in_channels = in_channels
        
        for i, kernel_size in enumerate(list(self.kernels)):
            sep_conv = nn.Conv1d(in_channels=in_channels, out_channels=self.planes, kernel_size=(kernel_size),
                               stride=1, padding=0, bias=False,)
            self.parallel_conv.append(sep_conv)

        self.bn1 = nn.BatchNorm1d(num_features=self.planes)
        self.relu = nn.ReLU(inplace=False)
        self.conv1 = nn.Conv1d(in_channels=self.planes, out_channels=self.planes, kernel_size=fixed_kernel_size,
                               stride=2, padding=2, bias=False)
        self.block = self._make_resnet_layer(kernel_size=fixed_kernel_size, stride=1, padding=fixed_kernel_size//2, drop_prob=drop_prob)
        self.bn2 = nn.BatchNorm1d(num_features=self.planes)
        self.avgpool = nn.AvgPool1d(kernel_size=6, stride=6, padding=2)
        self.rnn = nn.GRU(input_size=self.in_channels, hidden_size=128, num_layers=1, bidirectional=True)
        self.fc = nn.Linear(in_features=424, out_features=num_classes)

    def _make_resnet_layer(self, kernel_size, stride, blocks=9, padding=0, drop_prob=.1):
        layers = []
        downsample = None
        base_width = self.planes

        for i in range(blocks):
            downsampling = nn.Sequential(
                    nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
                )
            layers.append(ResNet_1D_Block(in_channels=self.planes, out_channels=self.planes, kernel_size=kernel_size,
                                       stride=stride, padding=padding, downsampling=downsampling, drop_prob=drop_prob))

        return nn.Sequential(*layers)
    def extract_features(self, x):
        # x : B=64 x T=10_000 x C=8
        x = x.permute(0, 2, 1)      # -> B x C x T
        out_sep = []

        for i in range(len(self.kernels)):
            sep = self.parallel_conv[i](x)
            out_sep.append(sep)

        out = torch.cat(out_sep, dim=2)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv1(out)  

        out = self.block(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.avgpool(out)  
        
        out = out.reshape(out.shape[0], -1)  
        rnn_out, _ = self.rnn(x.permute(0, 2, 1))
        new_rnn_h = rnn_out[:, -1, :]  
        

        new_out = torch.cat([out, new_rnn_h], dim=1) 
        return new_out
    
    def forward(self, x, targets=None):
        out = self.extract_features(x)
        logits = self.fc(out)  
        
        loss = None
        if targets is not None:
            loss = CFG.criterion(F.log_softmax(logits, dim=1), targets)

        return logits, loss

In [ ]:
import gc
iot = torch.randn(2, 10000, 8)#.cuda()
model = EEGNet(kernels=[3,5,7,9], in_channels=CFG.in_channels, fixed_kernel_size=5, num_classes=CFG.num_class, drop_prob=.2)
output, loss = model(iot)
print("{:.1f}M parameters".format(sum([p.numel() for p in model.parameters()])/1e6))
print(output.shape)

del iot, model
gc.collect()

In [ ]:
os.mkdir('./models')

In [ ]:

@torch.no_grad()
def eval(model, data_loader, eval_iters=5):
    model.eval()
    lossi = []
    for step, batch in enumerate(data_loader):
        eeg, labels = batch['eeg'].to(device), batch['labels'].to(device)
        _, loss = model(eeg, labels)
        if torch.cuda.device_count() > 1:
            loss = loss.mean()
        lossi.append(loss.item())
        if step > eval_iters:
            break
    return np.mean(lossi)


# A minimal training function
def train_loop(fold):

    train_folds = train[(train['fold'] != fold) & (train['total_evaluators'] >= 5)].reset_index(drop=True)
    valid_folds = train[(train['fold'] == fold)].reset_index(drop=True)

    train_dataset = EEGDataset(train_folds, CFG, mode="train")
    valid_dataset = EEGDataset(valid_folds, CFG, mode="train")

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    
    #==== MODEL ===
    model = EEGNet(kernels=[3,5,7,9], 
                   in_channels=CFG.in_channels, 
                   fixed_kernel_size=5, 
                   num_classes=CFG.num_class,
                   drop_prob=.1)
    model.to(device)
    model = nn.DataParallel(model)
    optim = torch.optim.AdamW(model.parameters())
    val_loss = 0.0

    for epoch in range(CFG.epochs):
        model.train()
        with tqdm(train_loader, unit='batches') as tepoch:
            for batch in tepoch:
                tepoch.set_description(f"Epoch {epoch+1}")
                eeg, labels = batch['eeg'].to(device), batch['labels'].to(device)
                optim.zero_grad()
                logits, loss = model(eeg, labels)
                
                if torch.cuda.device_count() > 1:
                    loss = loss.mean()
                
                loss.backward()
                optim.step()

                tepoch.set_postfix(loss=loss.item())
                
        print(f"After epoch {epoch+1}, val loss : {eval(model, valid_loader, eval_iters=10):.2f}")
    
    torch.save(model.state_dict(), f"./models/ver{CFG.version}_fold{fold}.pt")




In [ ]:
for fold in CFG.trn_fold:
    print(f"#========== Fold {fold} ==========")
    train_loop(fold)